In [1]:
import numpy as np

In [2]:
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize
import matplotlib.lines as mlines
from scipy.interpolate import UnivariateSpline
from tqdm import tqdm

In [3]:
from MCEq.core import MCEqRun
import mceq_config as config
import crflux.models as pm
from MCEq.data import InteractionCrossSections

In [4]:
import sys
import os

os.chdir('/hetghome/khymon/cs-analysis/')

# Import the class
from cs_modifier import ModIntCrossSections

In [5]:
# tune cross section

In [6]:
mceq_air = MCEqRun(
    interaction_model="SIBYLL23C",
    theta_deg=0.0,
    primary_model=(pm.GlobalSplineFitBeta, None),
    density_model = ('MSIS00_IC',('South Pole','January')),
)

MCEqRun::set_interaction_model(): SIBYLL23C
ParticleManager::_init_default_tracking(): Initializing default tracking categories (pi, K, mu)
MCEqRun::set_density_model(): Setting density profile to MSIS00_IC ('South Pole', 'January')
MSIS00IceCubeCentered::set_theta(): latitude = -90.00 for zenith angle =  0.00
MCEqRun::set_primary_model(): GlobalSplineFitBeta 


In [7]:
mceq_air.set_interaction_model("SIBYLL23C", force=True)
mceq_air.set_theta_deg(0)

MCEqRun::set_interaction_model(): SIBYLL23C
MCEqRun::set_primary_model(): GlobalSplineFitBeta 


In [8]:
cos_thetas = np.arange(0.8, 1.001, 0.1)
thetas = np.degrees(np.arccos(cos_thetas))
print(cos_thetas, thetas)

[0.8 0.9 1. ] [36.86989765 25.84193276  0.        ]


In [9]:
ptype = 'mu'
scale_factor_p = 1.4
scale_factor_k =1.
threshold = 1.e4
increase = 'const'
interactionmodel = "SIBYLL23C"

scale_factor = [scale_factor_p,scale_factor_k]

#initialize mceq instances
mceq_tune = MCEqRun(
    interaction_model="SIBYLL23C",
    theta_deg=0.0,
    primary_model=(pm.GlobalSplineFitBeta, None),
    density_model = (('MSIS00_IC',('SouthPole','January')))
)


# modify cross section
modcs = ModIntCrossSections(mceq_air._mceq_db, interaction_model=interactionmodel, scale_factor=scale_factor,threshold=threshold,increase=increase) #scale_factor=[1.,1.3],threshold=1.e4,increase='const')
modcs.load(interaction_model=interactionmodel)

mceq_tune._int_cs = modcs # add modification to cross section in mceq instance
mceq_tune.set_interaction_model(interactionmodel, force=True) # necessary to force cross section change

#test if tuning is correct:
print('ratio pion cross section tuned/untuned: ', modcs.get_cs(211, mbarn=True)/InteractionCrossSections(mceq_air._mceq_db, interaction_model=interactionmodel).get_cs(211, mbarn=True))
print('ratio kaon cross section tuned/untuned: ', modcs.get_cs(321, mbarn=True)/InteractionCrossSections(mceq_air._mceq_db, interaction_model=interactionmodel).get_cs(321, mbarn=True))

MCEqRun::set_interaction_model(): SIBYLL23C
ParticleManager::_init_default_tracking(): Initializing default tracking categories (pi, K, mu)
MCEqRun::set_density_model(): Setting density profile to MSIS00_IC ('SouthPole', 'January')
MSIS00IceCubeCentered::set_theta(): latitude = -90.00 for zenith angle =  0.00
MCEqRun::set_primary_model(): GlobalSplineFitBeta 
Applying mod with scaling factor: [1.3, 1.0], threshold e0: 11220.18454301964
Applying mod with scaling factor: [1.3, 1.0], threshold e0: 11220.18454301964
MCEqRun::set_interaction_model(): SIBYLL23C
Applying mod with scaling factor: [1.3, 1.0], threshold e0: 11220.18454301964
MCEqRun::set_primary_model(): GlobalSplineFitBeta 
ratio pion cross section tuned/untuned:  [nan nan nan nan nan nan nan nan 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.3 1.3 1.3
 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.

/tmp/ipykernel_817484/1680073817.py:27: RuntimeWarning: invalid value encountered in divide
  print('ratio pion cross section tuned/untuned: ', modcs.get_cs(211, mbarn=True)/InteractionCrossSections(mceq_air._mceq_db, interaction_model=interactionmodel).get_cs(211, mbarn=True))
/tmp/ipykernel_817484/1680073817.py:28: RuntimeWarning: invalid value encountered in divide
  print('ratio kaon cross section tuned/untuned: ', modcs.get_cs(321, mbarn=True)/InteractionCrossSections(mceq_air._mceq_db, interaction_model=interactionmodel).get_cs(321, mbarn=True))


In [10]:
seasons = ['January', 'April', 'July']
surface_flux_GSF = []

for season in tqdm(seasons):  # Loop over seasons
    season_flux = []  # Temporary container for the current season's results
    for ia, theta in enumerate(thetas):  # Loop over angles
        mceq_tune.set_theta_deg(theta)
        
        # Optionally set the season-dependent atmospheric model here
        mceq_tune.density_model.set_season(season)  # Uncomment and adjust if applicable
        
        # Solve for the current angle and season
        mceq_tune.solve()
        
        # Append the solution for the current angle
        season_flux.append(
            mceq_tune.get_solution("mu+", mag=0) + mceq_tune.get_solution("mu-", mag=0)
        )
    # Append the results for the current season
    surface_flux_GSF.append(season_flux)

  0%|          | 0/3 [00:00<?, ?it/s]

MSIS00IceCubeCentered::set_theta(): latitude = -90.00 for zenith angle = 36.87
MSIS00IceCubeCentered::set_theta(): latitude = -90.00 for zenith angle = 25.84
MSIS00IceCubeCentered::set_theta(): latitude = -90.00 for zenith angle =  0.00


 33%|███▎      | 1/3 [00:08<00:17,  8.71s/it]

MSIS00IceCubeCentered::set_theta(): latitude = -90.00 for zenith angle = 36.87
MSIS00IceCubeCentered::set_theta(): latitude = -90.00 for zenith angle = 25.84
MSIS00IceCubeCentered::set_theta(): latitude = -90.00 for zenith angle =  0.00


 67%|██████▋   | 2/3 [00:17<00:08,  8.71s/it]

MSIS00IceCubeCentered::set_theta(): latitude = -90.00 for zenith angle = 36.87
MSIS00IceCubeCentered::set_theta(): latitude = -90.00 for zenith angle = 25.84
MSIS00IceCubeCentered::set_theta(): latitude = -90.00 for zenith angle =  0.00


100%|██████████| 3/3 [00:26<00:00,  8.71s/it]


In [11]:
np.shape(surface_flux_GSF)

(3, 3, 121)

In [12]:
config.debug_level = 0

In [13]:
cr_grid = mceq_air.e_grid[30:-10] # cut because higher energies are not relevant for analysis

In [14]:
ground_muspec_prim_energies = []

for season in tqdm(seasons):  # Loop over seasons
    season_energies = []  # Temporary container for the current season's results
    for ei, eprim in enumerate(tqdm(cr_grid)):  # Loop over primary energies
        theta_energies = np.zeros((thetas.shape[0], mceq_air.dim))
        mceq_tune.set_single_primary_particle(E=eprim, pdg_id=2212)
        for ia, theta in enumerate(thetas):  # Loop over angles
            mceq_tune.set_theta_deg(theta)
            
            # Optionally set the season-dependent atmospheric model here
            mceq_tune.density_model.set_season(season)  # Uncomment and adjust if applicable
            
            mceq_tune.solve()
            theta_energies[ia, :] = mceq_tune.get_solution("mu+", mag=0) + mceq_tune.get_solution("mu-", mag=0)
        
        season_energies.append(theta_energies)
    # Append the results for the current season
    ground_muspec_prim_energies.append(season_energies)

100%|██████████| 3/3 [35:17<00:00, 705.88s/it]


In [15]:
import pickle

pickle.dump(
    [mceq_air.e_grid, cos_thetas, cr_grid, ground_muspec_prim_energies[0], ground_muspec_prim_energies[1], ground_muspec_prim_energies[2]],
    open("ground_muspec_prim_energies_season_cstune_pi" + str(scale_factor_p) + "_k" + str(scale_factor_k) + "_" +str(threshold)+ str(increase)+ ".pkl", "wb"),
)
pickle.dump(
    [mceq_air.e_grid, cos_thetas, surface_flux_GSF[0], surface_flux_GSF[1] , surface_flux_GSF[2]],
    open("surface_fluxes_season_pi" + str(scale_factor_p) + "_k" + str(scale_factor_k) + "_" +str(threshold)+ str(increase)+ ".pkl", "wb"),
)